# Character-Level Rnn

In [1]:
from io import open
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import unicodedata
import string


def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    try:
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
            and c in all_letters
        )
    except:
        return s

print(unicodeToAscii(u'Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print("Some arabic names ", category_lines["Arabic"][:10])
print("str of all letters", all_letters)

['data/names/Arabic.txt', 'data/names/Chinese.txt', 'data/names/Czech.txt', 'data/names/Dutch.txt', 'data/names/English.txt', 'data/names/French.txt', 'data/names/German.txt', 'data/names/Greek.txt', 'data/names/Irish.txt', 'data/names/Italian.txt', 'data/names/Japanese.txt', 'data/names/Korean.txt', 'data/names/Polish.txt', 'data/names/Portuguese.txt', 'data/names/Russian.txt', 'data/names/Scottish.txt', 'data/names/Spanish.txt', 'data/names/Vietnamese.txt']
Slusarski
('Some arabic names ', [u'Khoury', u'Nahas', u'Daher', u'Gerges', u'Nazari', u'Maalouf', u'Gerges', u'Naifeh', u'Guirguis', u'Baba'])
('str of all letters', "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'")


In [2]:
def letterToIndex(letter):
    return all_letters.find(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size()) # 1 is the batch size, 57 is the feature size and final axis 
                                    # is for input



Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     1     0     0     0

Columns 39 to 51 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 52 to 56 
    0     0     0     0     0
[torch.FloatTensor of size 1x57]

torch.Size([5, 1, 57])


In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
print(rnn)

RNN(
  (i2h): Linear(in_features=185, out_features=128)
  (i2o): Linear(in_features=185, out_features=18)
  (softmax): LogSoftmax()
)


In [4]:
input = Variable(lineToTensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input[0], hidden)
print(output)


Variable containing:

Columns 0 to 9 
-2.8463 -2.8522 -2.8140 -2.8626 -2.9689 -2.9459 -2.8350 -2.8607 -2.9405 -2.9855

Columns 10 to 17 
-2.7856 -2.9796 -2.9782 -2.8993 -2.9472 -2.8014 -2.8676 -2.8928
[torch.FloatTensor of size 1x18]



In [5]:
def categoryFromOutput(output):
    top_n, top_i = output.data.topk(1) # top output and it's index
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

print(categoryFromOutput(output))


('Japanese', 10)


In [6]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(lineToTensor(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

('category =', 'Korean', '/ line =', u'Li')
('category =', 'Czech', '/ line =', u'Neisser')
('category =', 'Greek', '/ line =', u'Close')
('category =', 'Czech', '/ line =', u'Weichert')
('category =', 'English', '/ line =', u'Rainey')
('category =', 'Greek', '/ line =', u'Paloumbas')
('category =', 'Chinese', '/ line =', u'Yim')
('category =', 'Chinese', '/ line =', u'Shum')
('category =', 'Japanese', '/ line =', u'Miyake')
('category =', 'Portuguese', '/ line =', u'Mendes')


### Training Network

In [7]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(rnn.parameters(), lr=0.005)

for epochs in range(100000):
    hidden = rnn.initHidden()
    rnn.zero_grad()

    category, line, category_tensor, line_tensor = randomTrainingExample()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()
    if epochs % 5000 == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = 'T' if guess == category else 'F (%s)' % category
        print('%d %d %% %.4f %s / %s %s' % (epochs, (epochs*100/ 100000), loss, line, guess, correct))


0 0 % 2.8202 Antonopoulos / Polish F (Greek)
5000 5 % 2.6244 Opova / Japanese F (Czech)
10000 10 % 2.4098 Hutton / Scottish F (English)
15000 15 % 3.2197 Demall / Italian F (Czech)
20000 20 % 1.3741 Van / Chinese F (Vietnamese)
25000 25 % 2.8789 Earl / Arabic F (English)
30000 30 % 2.6241 Imai / Arabic F (Japanese)
35000 35 % 1.4009 Pinho / Portuguese T
40000 40 % 3.3792 Pellar / Scottish F (Czech)
45000 45 % 0.6365 Vihorev / Russian T
50000 50 % 2.6355 Santiago / Japanese F (Spanish)
55000 55 % 0.2445 Yao / Chinese T
60000 60 % 1.3827 Sinclair / French F (Scottish)
65000 65 % 0.4310 Yang / Korean T
70000 70 % 0.5866 Pelaez / Spanish T
75000 75 % 1.7908 Sai / Chinese F (Vietnamese)
80000 80 % 2.9340 Sorge / English F (German)
85000 85 % 1.2986 Castro / Portuguese F (Italian)
90000 90 % 1.8282 Daele / French F (Dutch)
95000 95 % 1.6626 Ma / Korean F (Vietnamese)
